In [ ]:
import logging
import os
import requests
import json
from typing import Any, Callable, Set

# Azure AI Projects and authentication
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.identity import AzureCliCredential

# Azure Blob Storage client
from azure.storage.blob import BlobServiceClient

# Load environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()
import datetime

# Retrieve keys from environment variables
BING_API_KEY = os.getenv("BING_API_KEY")
PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

if not BING_API_KEY:
    raise ValueError("BING_API_KEY is not set in the .env file.")
if not PROJECT_CONNECTION_STRING:
    raise ValueError("PROJECT_CONNECTION_STRING is not set in the .env file.")
if not AZURE_STORAGE_CONNECTION_STRING:
    raise ValueError("AZURE_STORAGE_CONNECTION_STRING is not set in the .env file.")




def search_for_relevant_news(query: str = "Artificial Intelligence", news_count: int = 10) -> str:
    """
    Uses the Bing Search API to fetch news articles related to the query.
    
    Args:
        query (str): The search term. Defaults to "Artificial Intelligence".
        news_count (int): Number of articles to return. Defaults to 10.
    
    Returns:
        A JSON-formatted string containing a list of news articles (title and URL).
    """
    logging.info(f"Searching for news articles related to: {query}")
    endpoint = "https://api.bing.microsoft.com/v7.0/news/search"
    params = {"q": query, "mkt": "en-US", "count": news_count}
    headers = {"Ocp-Apim-Subscription-Key": BING_API_KEY}

    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        news_list = []
        for item in data.get("value", []):
            news_entry = {
                "title": item.get("name", "No Title"),
                "url": item.get("url", "No URL"),
            }
            news_list.append(news_entry)
        return json.dumps(news_list, indent=4)
    except Exception as e:
        logging.error(f"Error calling Bing Search API: {e}")
        return json.dumps({"error": str(e)})


def store_thought_process_in_blob(thought_summary: str, container_name: str = "thoughts") -> str:
    """
    Uploads the agent's thought process summary to an Azure Blob Storage container.

    Args:
        thought_summary (str): A text summary of the agent's thought process.
        container_name (str): Name of the container in Blob Storage.

    Returns:
        str: A message indicating success or failure.
    """
    try:
        # Create a BlobServiceClient using the storage connection string.
        blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
        # Get the container client.
        container_client = blob_service_client.get_container_client(container_name)

        # Create the container if it does not exist.
        try:
            container_client.create_container()
            logging.info(f"Container '{container_name}' created.")
        except Exception as e:
            logging.info(f"Container '{container_name}' may already exist: {e}")

        # Create a timestamp for a unique file name.
        timestamp = datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")
        blob_name = f"thoughts_{timestamp}.txt"

        # Get a blob client and upload the text data.
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(thought_summary, overwrite=True)
        success_message = f"Thought process successfully stored in blob storage: {container_name}/{blob_name}"
        logging.info(success_message)
        return success_message
    except Exception as e:
        error_message = f"Error uploading thought process to blob storage: {e}"
        logging.error(error_message)
        return error_message
    

def store_news_in_blob(news_json: str, container_name: str = "news") -> str:
    """
    Uploads the provided news JSON data to an Azure Blob Storage container and returns a success or error message.

    Args:
        news_json (str): JSON formatted string containing news data.
        container_name (str): Name of the container in Blob Storage.

    Returns:
        str: A message indicating success or failure.
    """
    try:
        # Create a BlobServiceClient using the storage connection string.
        blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
        # Get the container client.
        container_client = blob_service_client.get_container_client(container_name)

        # Create the container if it does not exist.
        try:
            container_client.create_container()
            logging.info(f"Container '{container_name}' created.")
        except Exception as e:
            logging.info(f"Container '{container_name}' may already exist: {e}")

        # Create a timestamp for a unique file name.
        timestamp = datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")
        blob_name = f"news_{timestamp}.json"

        # Get a blob client and upload the JSON data.
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(news_json, overwrite=True)
        success_message = f"News data successfully stored in blob storage: {container_name}/{blob_name}"
        logging.info(success_message)
        return success_message
    except Exception as e:
        error_message = f"Error uploading news data to blob storage: {e}"
        logging.error(error_message)
        return error_message

# ---------------------------------------------------------------------------
# Prepare the tool for the Azure AI Agent service.
# We wrap our search function in a FunctionTool and add it to a ToolSet.
# ---------------------------------------------------------------------------

user_functions: Set[Callable[..., Any]] = {
    search_for_relevant_news,
    store_news_in_blob,
    store_thought_process_in_blob
}
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

def run_agent():
    """
    Runs the Azure AI Agent pipeline.
    
    The agent uses the Bing Search API tool to fetch the latest AI news.
    It creates a simple conversation thread where the user message triggers the tool.
    """
    logging.info("Starting the Azure AI Agent pipeline...")
    # Create the Azure AI Projects client with a connection string and default credential.
    client = AIProjectClient.from_connection_string(
        credential=AzureCliCredential(),
        conn_str=PROJECT_CONNECTION_STRING,
        
    )
    
    # Instructions for the agent
    instructions = """
    You are a helpful assistant that provides the latest news articles about AI.
    Always se the 'search_for_relevant_news' function to fetch and return current AI news.
    Then store the news data in Azure Blob Storage using the 'store_news_in_blob' function.
    At the end, summarize your thought process and store it in Azure Blob Storage using the 'store_thought_process_in_blob' function.
    """
    
    with client:
        # Create the agent using a chosen model (e.g., gpt-4o-mini)
        agent = client.agents.create_agent(
            model="gpt-4o-mini",
            name="simple-news-agent",
            instructions=instructions,
            toolset=toolset
        )
        logging.info(f"Created agent with ID: {agent.id}")

        # Start a new conversation thread
        thread = client.agents.create_thread()
        logging.info(f"Created thread with ID: {thread.id}")

        # Create an initial user message
        message = client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="Show me the latest AI news."
        )
        logging.info(f"User message created with ID: {message.id}")

        # Process the conversation
        run = client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        logging.info(f"Run finished with status: {run.status}")

        if run.status == "failed":
            logging.error(f"Run failed: {run.last_error}")
        else:
            # Retrieve and log all messages from the conversation
            messages = client.agents.list_messages(thread_id=thread.id)
            for msg in messages["data"]:
                if msg["role"] == "assistant":
                    logging.info("Assistant response:")
                    for part in msg["content"]:
                        if part["type"] == "text":
                            logging.info(part["text"]["value"])

        # Cleanup the agent after the run
        client.agents.delete_agent(agent.id)
        logging.info("Agent deleted successfully.")


run_agent()

2025-02-25 14:11:07,185 - INFO - Starting the Azure AI Agent pipeline...
2025-02-25 14:11:08,195 - INFO - AzureCliCredential.get_token succeeded
2025-02-25 14:11:08,200 - INFO - Request URL: 'https://swedencentral.api.azureml.ms/agents/v1.0/subscriptions/4c9216b8-3c30-4c2f-8ced-0837fea45954/resourceGroups/basic-agent-setup-713/providers/Microsoft.MachineLearningServices/workspaces/ffollonier-rag-project-713/assistants?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1144'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'f95b48b9-f379-11ef-ab51-8cc681cfef94'
    'User-Agent': 'azsdk-python-ai-projects/1.0.0b5 Python/3.11.5 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
2025-02-25 14:11:08,943 - INFO - Response status: 200
Response headers:
    'Date': 'Tue, 25 Feb 2025 13:11:09 GMT'
    'Content-Type': 'application/json'
    'Transfer-Encoding': 'chunke